# Obtain mean head pose on *trainval* split and use as predictions for test dataset
Implicitly documents test submission format

In [ ]:
import os

from dd_pose.dataset import Dataset
from dd_pose.dataset_item import DatasetItem
from dd_pose.image_decorator import ImageDecorator
from dd_pose.jupyter_helpers import showimage
from dd_pose.evaluation_helpers import T_headfrontal_camdriver, T_camdriver_headfrontal
import transformations as tr
import json

import numpy as np

In [ ]:
class MeanPredictor:
    def __init__(self):
        self.mean_T_camdriver_head = None
        
    def get_name(self):
        return 'Prior'
    
    def get_dirname(self):
        return 'prior'
    
    def get_metadata(self):
        return dict()
    
    def initialize_from_dataset(self, dataset):
        # get mean translation and rotation across the whole dataset
        # we compute roll, pitch, yaw wrt. 'headfrontal' frame in order to avoid gimbal lock averaging later on
        xyzs = []
        rpys = []

        for di_dict in dataset.get_dataset_items():
            di = DatasetItem(di_dict)
            print(di)
            for stamp in di.get_stamps():
                T_camdriver_head = di.get_T_camdriver_head(stamp)
                T_headfrontal_head = np.dot(T_headfrontal_camdriver, T_camdriver_head)
                rpy = tr.euler_from_matrix(T_headfrontal_head)
                rpys.append(rpy)
                xyzs.append(T_camdriver_head[0:3,3])
                
        # rpy mean in headfrontal frame
        mean_rpy = np.mean(np.array(rpys), axis=0)
        print(mean_rpy)
        # xyz mean in camdriver frame
        mean_xyz = np.mean(np.array(xyzs), axis=0)
        print(mean_xyz)

        # rotational component from mean rpy to camdriver frame
        mean_T_headfrontal_head = tr.euler_matrix(*mean_rpy)
        self.mean_T_camdriver_head = np.dot(T_camdriver_headfrontal, mean_T_headfrontal_head)
        # translational component from mean xyz in camdriver frame
        self.mean_T_camdriver_head[0:3,3] = mean_xyz
    
    def get_T_camdriver_head(self, stamp):
        return self.mean_T_camdriver_head


In [ ]:
# initialize mean predictor with measurements from trainval split
mean_predictor = MeanPredictor()
mean_predictor.initialize_from_dataset(Dataset(split='trainval'))

In [ ]:
mean_predictor.get_T_camdriver_head(0)

# Draw mean head pose onto sample image

In [ ]:
d = Dataset(split='trainval')
di_dict = d.get(subject_id=1, scenario_id=3, humanhash='sodium-finch-fillet-spring')
di = DatasetItem(di_dict)
print(di)

In [ ]:
stamp = di.get_stamps()[50]
stamp

In [ ]:
img, pcm = di.get_img_driver_left(stamp, shift=True)
img_bgr = np.dstack((img, img, img))

image_decorator = ImageDecorator(img_bgr, pcm)
image_decorator.draw_axis(mean_predictor.get_T_camdriver_head(stamp), use_gray=False)
image_decorator.draw_axis(di.get_T_camdriver_head(stamp), use_gray=True)

colored axis: mean head pose
gray axis: measurement (ground truth)

In [ ]:
showimage(img_bgr)

In [ ]:
prediction_output_base_dir = os.path.join(os.environ['DD_POSE_DATA_ROOT_DIR'], '10-predictions')
try:
    os.makedirs(prediction_output_base_dir)
except:
    pass

# Write out mean head pose predictions for test dataset

In [ ]:
d = Dataset(split='test')
predictor = mean_predictor
predictor.get_name(), predictor.get_dirname()

In [ ]:
predictor_predictions_dir = os.path.join(prediction_output_base_dir, predictor.get_dirname())
assert not os.path.exists(predictor_predictions_dir), "Predictions already written out. Aborting. %s" % predictor_predictions_dir

In [ ]:
for di_dict in d.get_dataset_items():
    di = DatasetItem(di_dict)
    print(di)
    
    predictions_dir = os.path.join(predictor_predictions_dir, 'subject-%02d' % di.get_subject(), 'scenario-%02d' % di.get_scenario(), di.get_humanhash())
    
    try:
        os.makedirs(predictions_dir)
    except OSError as e:
        pass
        
    predictions = dict()
    for stamp in di.get_stamps():
        predictions[stamp] = predictor.get_T_camdriver_head(stamp).tolist()
    
    # write out predictions
    with open(os.path.join(predictions_dir, 't-camdriver-head-predictions.json'), 'w') as fp:
        json.dump(predictions, fp, sort_keys=True, indent=4)
        
    metadata = {
        'name': predictor.get_name(),
        'dirname': predictor.get_dirname(),
        'metadata': predictor.get_metadata()
    }
    with open(os.path.join(predictor_predictions_dir, 'metadata.json'), 'w') as fp:
        json.dump(metadata, fp, sort_keys=True, indent=4)

In [ ]:
# now tar.gz the predictions in the format expected by the benchmark website
print('pushd %s; tar czf %s subject-* metadata.json; popd' % (predictor_predictions_dir,\
                            os.path.join(predictor_predictions_dir, 'predictions.tar.gz')))